# Tools in LangGraph: External Function Integration

Tools in LangGraph are external functions or services that can be called from within your graph nodes. They enable your graph to interact with external systems, APIs, databases, and other services, making your workflows more powerful and connected.

## Key Features:
- **External Integration**: Connect to external services and APIs
- **Function Calls**: Execute external functions from graph nodes
- **Data Processing**: Process data using external tools
- **Service Orchestration**: Coordinate multiple external services

## Types of Tools:

### **1. API Tools**
```python
import requests

def weather_api_tool(location: str) -> dict:
    """Get weather data from external API"""
    try:
        response = requests.get(f"https://api.weather.com/v1/current?location={location}")
        return {
            "success": True,
            "temperature": response.json()["temperature"],
            "condition": response.json()["condition"]
        }
    except Exception as e:
        return {"success": False, "error": str(e)}
```

### **2. Database Tools**
```python
import sqlite3

def database_query_tool(query: str) -> dict:
    """Execute database query"""
    try:
        conn = sqlite3.connect("app.db")
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.close()
        return {"success": True, "data": results}
    except Exception as e:
        return {"success": False, "error": str(e)}
```

### **3. File Processing Tools**
```python
import os
import json

def file_processor_tool(file_path: str, operation: str) -> dict:
    """Process files based on operation type"""
    try:
        if operation == "read":
            with open(file_path, 'r') as f:
                content = f.read()
            return {"success": True, "content": content}
        elif operation == "write":
            # Write operation logic
            return {"success": True, "message": "File written successfully"}
        else:
            return {"success": False, "error": "Unknown operation"}
    except Exception as e:
        return {"success": False, "error": str(e)}
```

## Tool Integration Patterns:

### **1. Direct Tool Calls**
```python
def process_with_tool(state):
    """Node that calls external tool directly"""
    user_input = state.get("user_input", "")
    
    # Call external tool
    weather_data = weather_api_tool(user_input)
    
    return {
        "weather_data": weather_data,
        "processed": True
    }
```

### **2. Tool with Error Handling**
```python
def safe_tool_call(state):
    """Node with robust tool error handling"""
    try:
        # Call external tool
        result = external_api_tool(state.get("query"))
        
        if result.get("success"):
            return {"data": result["data"], "status": "success"}
        else:
            return {"error": result.get("error"), "status": "failed"}
    
    except Exception as e:
        return {"error": f"Tool call failed: {str(e)}", "status": "error"}
```

### **3. Conditional Tool Usage**
```python
def conditional_tool_usage(state):
    """Use different tools based on conditions"""
    query_type = state.get("query_type", "unknown")
    
    if query_type == "weather":
        result = weather_api_tool(state.get("location"))
    elif query_type == "news":
        result = news_api_tool(state.get("topic"))
    else:
        result = general_search_tool(state.get("query"))
    
    return {"tool_result": result}
```

## Advanced Tool Patterns:

### **Tool Chaining**
```python
def chain_tools(state):
    """Chain multiple tools together"""
    # First tool: Get data
    data_result = data_fetch_tool(state.get("query"))
    
    if data_result.get("success"):
        # Second tool: Process data
        processed_result = data_processor_tool(data_result["data"])
        
        if processed_result.get("success"):
            # Third tool: Save results
            save_result = save_tool(processed_result["data"])
            return {"final_result": save_result}
    
    return {"error": "Tool chain failed"}
```

### **Parallel Tool Execution**
```python
import asyncio
import aiohttp

async def parallel_tools(state):
    """Execute multiple tools in parallel"""
    tasks = [
        weather_api_tool(state.get("location")),
        news_api_tool(state.get("topic")),
        stock_api_tool(state.get("symbol"))
    ]
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    return {
        "weather": results[0],
        "news": results[1],
        "stocks": results[2]
    }
```

### **Tool with Retry Logic**
```python
import time

def retry_tool_call(state, max_retries=3):
    """Tool call with retry logic"""
    for attempt in range(max_retries):
        try:
            result = external_api_tool(state.get("query"))
            if result.get("success"):
                return result
        except Exception as e:
            if attempt == max_retries - 1:
                return {"success": False, "error": str(e)}
            time.sleep(2 ** attempt)  # Exponential backoff
    
    return {"success": False, "error": "Max retries exceeded"}
```

## Tool State Management:

### **Tool Results in State**
```python
def tool_with_state_management(state):
    """Manage tool results in state"""
    # Call tool
    tool_result = external_tool(state.get("input"))
    
    # Update state with tool results
    return {
        "tool_success": tool_result.get("success", False),
        "tool_data": tool_result.get("data"),
        "tool_error": tool_result.get("error"),
        "execution_time": time.time() - state.get("start_time", 0)
    }
```

### **Tool Configuration**
```python
def configurable_tool(state):
    """Tool with configuration from state"""
    config = state.get("tool_config", {})
    
    # Use configuration for tool call
    result = configured_api_tool(
        endpoint=config.get("endpoint", "default"),
        timeout=config.get("timeout", 30),
        retries=config.get("retries", 3)
    )
    
    return {"tool_result": result}
```

## Common Tool Categories:

### **Data Tools**
- Database queries
- File operations
- Data transformation
- Data validation

### **API Tools**
- REST API calls
- GraphQL queries
- Webhook triggers
- Service integration

### **AI/ML Tools**
- Model inference
- Data preprocessing
- Feature extraction
- Prediction services

### **Utility Tools**
- Email sending
- SMS notifications
- Logging services
- Monitoring tools

## Best Practices:

### **1. Error Handling**
- Always handle tool failures gracefully
- Provide meaningful error messages
- Implement retry logic for transient failures
- Log tool execution details

### **2. Performance**
- Use async tools for I/O operations
- Implement caching for expensive tools
- Set appropriate timeouts
- Monitor tool execution times

### **3. Security**
- Validate tool inputs
- Use secure authentication
- Handle sensitive data properly
- Implement access controls

### **4. Testing**
- Mock external tools in tests
- Test error scenarios
- Validate tool outputs
- Monitor tool health

## Tool Integration Examples:

### **Weather Service Integration**
```python
def weather_workflow(state):
    """Complete weather workflow with tools"""
    location = state.get("location")
    
    # Get weather data
    weather = weather_api_tool(location)
    
    # Process weather data
    processed = weather_processor_tool(weather)
    
    # Send notification
    notification = notification_tool(processed)
    
    return {
        "weather_data": weather,
        "processed_data": processed,
        "notification_sent": notification
    }
```

### **Data Pipeline Tools**
```python
def data_pipeline(state):
    """Data pipeline with multiple tools"""
    # Extract data
    raw_data = data_extraction_tool(state.get("source"))
    
    # Transform data
    transformed = data_transformation_tool(raw_data)
    
    # Load data
    loaded = data_loading_tool(transformed)
    
    # Generate report
    report = report_generation_tool(loaded)
    
    return {
        "pipeline_status": "completed",
        "data_processed": len(transformed),
        "report_generated": report
    }
```

## Benefits:
- **External Integration**: Connect to any external service
- **Modularity**: Reusable tool functions
- **Flexibility**: Easy to add new capabilities
- **Scalability**: Handle complex external operations
- **Maintainability**: Centralized tool management

## Tips for Success:
- Design tools to be stateless when possible
- Use consistent error handling patterns
- Document tool interfaces clearly
- Implement proper logging and monitoring
- Test tools independently
- Consider rate limits and quotas
- Plan for tool failures and fallbacks
